In [26]:
import pandas as pd

playlist_data = pd.read_csv('./spotify_data/playlistCSV.csv', header=0)
streaming_history = pd.read_csv('./spotify_data/streamingHistoryCSV.csv', header=0)
playlist_data.head()

,albumName,artistName,trackName,trackUri,playlists.name
0,...Like Clockwork,Queens of the Stone Age,I Sat by the Ocean,spotify:track:1D3YGmST01Ba0WRiT7THPG,The Knew
1,¡DOS!,Green Day,Stray Heart,spotify:track:2Q22sa3Y4lKXHHBBAxEhYY,Missing Her
2,¡DOS!,Green Day,Stray Heart,spotify:track:2Q22sa3Y4lKXHHBBAxEhYY,Perfect Playlist Pt 2
3,¡UNO!,Green Day,Nuclear Family,spotify:track:206n37mZTmloL45RYhmMlP,Missing Her
4,¡UNO!,Green Day,Nuclear Family,spotify:track:206n37mZTmloL45RYhmMlP,Rock Paper Scissors


In [27]:
merged_dataset = streaming_history.merge(playlist_data, on='trackName',)
merged_dataset['time'] = merged_dataset['time'].str.split(' ').str[1]
merged_dataset['track_id'] = merged_dataset['trackUri'].str.split('track:').str[1]
merged_dataset.to_csv('./cleaned_data/mergedSpotifyData.csv', index=False)


In [33]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

client_id = '3811b236a5dd410e9f8bb2321bbdef9f'
client_secret = '5daa963dee6d41dd820c8d2a776c1fcc'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [34]:
requests_per_minute = 150
requests_per_hour = 9000

for index, track_uri in enumerate(merged_dataset['trackUri']):
    audio_features = sp.audio_features(track_uri)

    time.sleep(30)

    if index % requests_per_minute == 0 and index != 0:
        time_to_sleep = 60
        time.sleep(time_to_sleep)
        index = 0

    if index % requests_per_hour == 0 and index != 0:
        time_to_sleep = 3600
        time.sleep(time_to_sleep)
        index = 0

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5r96TaQquRrlo3Ym3ZlSL2:
 Max Retries, reason: too many 429 error responses

In [30]:
api_dataset = pd.DataFrame(audio_features)
columns_to_drop = ['analysis_url','track_href','type','uri']
api_dataset.drop(columns_to_drop, axis=1,inplace=True)
api_dataset.rename(columns={'id': 'track_id'}, inplace=True)


In [31]:
spotify_dataset = merged_dataset.merge(audio_features, on='track_id')
columns_to_drop = ['trackUri','Sum of msPlayed','artistName_x']
spotify_dataset.drop(columns_to_drop, axis=1,inplace=True)
spotify_dataset.rename(columns={'artistName_y': 'artistName'}, inplace=True)
spotify_dataset.to_csv('apiData.csv', index=False)

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed